<a href="https://colab.research.google.com/github/fmcarvjal/6.canvas-app/blob/master/wisper__pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio gcsfs fsspec==2024.10.0


INFO: pip is looking at multiple versions of datasets[audio] to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
INFO: pip is still looking at multiple versions of datasets[audio] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dil

In [ ]:
!pip show fsspec

Name: fsspec
Version: 2024.10.0
Summary: File-system specification
Home-page: https://github.com/fsspec/filesystem_spec
Author: 
Author-email: 
License: BSD 3-Clause License

Copyright (c) 2018, Martin Durant
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRES

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
import gradio as gr
import torch

from datasets import Audio, DatasetDict, load_dataset
from huggingface_hub import notebook_login
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    WhisperFeatureExtractor,
    WhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperTokenizer,
    pipeline,
)

In [ ]:
notebook_login()

In [ ]:
!rm -rf ~/.cache/huggingface/datasets

In [ ]:
dataset_name = "mozilla-foundation/common_voice_11_0"
dataset = DatasetDict()

dataset["train"] = load_dataset(
    dataset_name,
    "hi",
    split="train+validation",
    download_mode="force_redownload",
    keep_in_memory=True,
    cache_dir="/content/datasets_cache"
).select(range(50))

dataset["test"] = load_dataset(
    dataset_name,
    "hi",
    split="test",
    download_mode="force_redownload",
    keep_in_memory=True,
    cache_dir="/content/datasets_cache"
).select(range(25))

print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

...

Generating other split:   0%|          | 0/3328 [00:00<?, ? examples/s]


Reading metadata...: 3328it [00:00, 93909.70it/s]


Generating invalidated split:   0%|          | 0/680 [00:00<?, ? examples/s]


Reading metadata...: 680it [00:00, 27609.33it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 50
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 25
    })
})


In [ ]:
dataset = dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 50
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 25
    })
})


In [ ]:
model_name = "openai/whisper-small"
language = "hi"

preprocessor = WhisperProcessor.from_pretrained(model_name, language=language, task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language=language, task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name)

model.generation_config.language = language
model.generation_config.task = "transcribe"

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
def preprocess_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

dataset = dataset.map(preprocess_dataset, remove_columns=dataset.column_names["train"])

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Configurar el modelo Whisper para el entrenamiento

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features):

        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)


        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=preprocessor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_name}-hi",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs=1,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=500,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Entrenar el modelo

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=preprocessor.feature_extractor,
)

<ipython-input-10-6cf083193e43>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
metric = evaluate.load("wer")

trainer.train()

Epoch,Training Loss,Validation Loss,Wer
1,No log,1.015178,63.636364


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=2, training_loss=0.8993260860443115, metrics={'train_runtime': 2694.8575, 'train_samples_per_second': 0.019, 'train_steps_per_second': 0.001, 'total_flos': 1.4429270016e+16, 'train_loss': 0.8993260860443115, 'epoch': 1.0})

In [ ]:
kwargs = {
    "dataset_tags": dataset_name,
    "dataset": f"{dataset_name} {language}",
    "dataset_args": "config: {language}, split: test",
    "language": language,
    "model_name": f"{model_name} - OpenWebinars",  # Uso de f-string
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)


NameError: name 'dataset_name' is not defined